### 0. 모델 로드 및 전처리 등 실험과 상관 없는 내용

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models,datasets, transforms
from mevit_model import MultiExitViT
from tqdm import tqdm
from itertools import combinations
import numpy as np
from scipy.stats import entropy
from temperature_scaling import TemperatureScaling
from Dloaders import Dloaders
####################################################################
IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name = {'cifar10':datasets.CIFAR10, 'cifar100':datasets.CIFAR100,'imagenet':None}
dataset_outdim = {'cifar10':10, 'cifar100':100,'imagenet':1000}
##############################################################
################ 0. Hyperparameters ##########################
##############################################################
batch_size = 1024
data_choice='cifar10'
mevit_isload=True
mevit_pretrained_path=f"models/{data_choice}/integrated_ee.pth"

backbone_path=f'models/{data_choice}/vit_{data_choice}_backbone.pth'
start_lr=1e-4
max_iter=200

ee_list=[0,1,2,3,4,5,6,7,8,9]#exit list ex) [0,1,2,3,4,5,6,7,8,9]
exit_loss_weights=[1,1,1,1,1,1,1,1,1,1,1]#exit마다 가중치
exit_num=11
##############################################################
dloaders=Dloaders(data_choice=data_choice,batch_size=batch_size,IMG_SIZE=IMG_SIZE)
train_loader,test_loader = dloaders.get_loaders()

# Load the pretrained ViT model from the saved file
pretrained_vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
if data_choice != 'imagenet':
    pretrained_vit.heads.head = nn.Linear(pretrained_vit.heads.head.in_features, dataset_outdim[data_choice])  # Ensure output matches the number of classes

    # Load model weights
    pretrained_vit.load_state_dict(torch.load(backbone_path))
    pretrained_vit = pretrained_vit.to(device)
#from torchinfo import summary
#summary(pretrained_vit,input_size= (64, 3, IMG_SIZE, IMG_SIZE))

model = MultiExitViT(pretrained_vit,num_classes=dataset_outdim[data_choice],ee_list=ee_list,exit_loss_weights=exit_loss_weights).to(device)

# Assume a pretrained model (replace with your own model)
model.load_state_dict(torch.load(mevit_pretrained_path))  # Load your trained weights
file_path = f'cache_result_mevit_{data_choice}.pt'

Files already downloaded and verified
Files already downloaded and verified


### 1. MEVIT의 각 출구들에서의 Precision을 확인

In [ ]:
cifar10_accs = [72.76, 80.35, 85.8, 89.42, 91.98, 93.12, 94.94, 96.44, 96.95, 97.35, 97.63]
cifar100_accs = [48.21, 59.63, 67.93, 73.31, 77.9, 80.8, 83.71, 85.33, 86.61, 87.04, 87.75]
imagenet_accs = [34.76, 42.65, 51.82, 57.76, 62.19, 65.5, 69.29, 72.3, 75.3, 77.6, 81.06]

### 2. MEVIT의 각 출구들까지의 FLOPs 확인

In [ ]:
cifar10_FLOPs_list = [2908083456, 4303940352, 5699797248, 7095654144, 8491511040, 9887367936, 11283224832, 12679081728, 14074938624, 15470795520, 16866652416]
cifar100_FLOPs_list = [2908152576, 4304009472, 5699866368, 7095723264, 8491580160, 9887437056, 11283293952, 12679150848, 14075007744, 15470864640, 16866721536]
#imagenet_FLOPs_list=

In [2]:
from fvcore.nn import FlopCountAnalysis
x = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
flops = FlopCountAnalysis(model, x)
print(flops.total())

Unsupported operator aten::mul encountered 89 time(s)
Unsupported operator aten::add encountered 45 time(s)
Unsupported operator aten::div encountered 22 time(s)
Unsupported operator aten::unflatten encountered 22 time(s)
Unsupported operator aten::scaled_dot_product_attention encountered 22 time(s)
Unsupported operator aten::gelu encountered 22 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
base_model.encoder.layers.encoder_layer_0.self_attention.out_proj, base_model.encoder.layers.encoder_layer_1.self_attention.out_proj, base_model.encoder.layers.encoder_layer_10.self_attention.out_proj, base_model.encoder.layers.encoder_layer_11.self_attention.out_proj, base_model.encoder.layers.encoder_layer_2.self_atten

30832862976


In [ ]:
'''from torchinfo import summary    #ViT는 summary FLOPs 계산이 100분의 1배로 계산나옴...
tmp=[]
for i in range(exit_num):
    model.set_each_ee_test_mode(i)
    #summary(model,input_size= (1, 3, IMG_SIZE, IMG_SIZE))
    tmp.append(summary(model,input_size= (1, 3, IMG_SIZE, IMG_SIZE)).total_mult_adds)
tmp''' 

tmp = []
for i in range(exit_num):
    model.set_each_ee_test_mode(i)
    x = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
    flops = FlopCountAnalysis(model, x)
    tmp.append(flops.total())
tmp

Unsupported operator aten::mul encountered 9 time(s)
Unsupported operator aten::add encountered 5 time(s)
Unsupported operator aten::div encountered 2 time(s)
Unsupported operator aten::unflatten encountered 2 time(s)
Unsupported operator aten::scaled_dot_product_attention encountered 2 time(s)
Unsupported operator aten::gelu encountered 2 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
base_model.encoder.layers.encoder_layer_0.self_attention.out_proj, base_model.encoder.layers.encoder_layer_1, base_model.encoder.layers.encoder_layer_1.dropout, base_model.encoder.layers.encoder_layer_1.ln_1, base_model.encoder.layers.encoder_layer_1.ln_2, base_model.encoder.layers.encoder_layer_1.mlp, base_model.encoder.layer

[2908083456,
 4303940352,
 5699797248,
 7095654144,
 8491511040,
 9887367936,
 11283224832,
 12679081728,
 14074938624,
 15470795520,
 16866652416]

In [ ]:
# ResNet101 FLOPs: 16866721536
# 7799667684(resnet101) VS 16866721536(ViT-b/16)
from torchinfo import summary
pretrained_resnet101 = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
pretrained_resnet101.fc = nn.Linear(pretrained_resnet101.fc.in_features, dataset_outdim[data_choice])
summary(pretrained_resnet101,input_size= (1, 3, IMG_SIZE, IMG_SIZE)).total_mult_adds

7799667684

In [10]:
print(16866721536 / 7799667684 * 100,'%') #ViT-b/16의 flops가 resnet101의 216.24%이다.. 즉 2배이다.

216.24923290770295 %


In [ ]:
# 만약 앙상블을 한다면 총 연산량은 단일출구에 비해 1.8배가 될 것이다.
print(30832862976 / 16866721536 * 100,'%')

182.80294075046498 %
